In [20]:
!pip install -q ultralytics
import os
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt


In [21]:
root_dir = '/kaggle/input/dataset256/UECFOOD256'
out_img_dir = '/kaggle/working/dataset/images'
out_lbl_dir = '/kaggle/working/dataset/labels'
for split in ['train', 'val', 'test']:
    os.makedirs(f'{out_img_dir}/{split}', exist_ok=True)
    os.makedirs(f'{out_lbl_dir}/{split}', exist_ok=True)

image_label_pairs = []
skipped = 0

for class_id in range(1, 257):
    class_path = os.path.join(root_dir, str(class_id))
    bbox_file = os.path.join(class_path, "bb_info.txt")

    if not os.path.exists(bbox_file):
        continue

    with open(bbox_file, 'r') as f:
        lines = f.readlines()[1:]

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            skipped += 1
            continue

        img_name, x1, y1, x2, y2 = parts
        try:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        except:
            skipped += 1
            continue

        img_path = os.path.join(class_path, f"{img_name}.jpg")
        if not os.path.exists(img_path):
            skipped += 1
            continue

        try:
            with Image.open(img_path) as img:
                w, h = img.size
        except:
            skipped += 1
            continue

        xc = ((x1 + x2) / 2) / w
        yc = ((y1 + y2) / 2) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h

        new_name = f"{class_id}_{img_name}.jpg"
        label = class_id - 1
        image_label_pairs.append((img_path, new_name, label, xc, yc, bw, bh))

print(f"✅ Total usable images: {len(image_label_pairs)}, Skipped: {skipped}")


✅ Total usable images: 31645, Skipped: 0


In [22]:
train, temp = train_test_split(image_label_pairs, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

def save_data(split, name):
    for img_path, fname, cls, xc, yc, bw, bh in split:
        shutil.copy(img_path, f"{out_img_dir}/{name}/{fname}")
        with open(f"{out_lbl_dir}/{name}/{fname.replace('.jpg', '.txt')}", "w") as f:
            f.write(f"{cls} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

save_data(train, 'train')
save_data(val, 'val')
save_data(test, 'test')


In [23]:
# cat_map = {}
# with open('/kaggle/input/dataset256/UECFOOD256/category.txt', 'r') as f:
#     for line in f:
#         try:
#             idx, name = line.strip().split(maxsplit=1)
#             cat_map[int(idx) - 1] = name.strip()
#         except:
#             continue

# Now write dataset.yaml
# with open("dataset.yaml", "w") as f:
#     f.write("path: /kaggle/working/dataset\n")
#     f.write("train: images/train\n")
#     f.write("val: images/train\n")
#     f.write("names:\n")
#     for i in range(256):
#         f.write(f"  {i}: {cat_map.get(i, f'food_{i}')}\n")

with open('/kaggle/working/dataset.yaml', 'w') as f:
    f.write(f"""
path: /kaggle/working/dataset
train: images/train
val: images/val
test: images/test
nc: 256
names: [{', '.join([f'"class_{i}"' for i in range(256)])}]
""")


In [24]:
from ultralytics import YOLO

# Load pre-trained YOLOv8n (Nano) model — fastest for testing
# model = YOLO('yolov8n.pt')  # you can try yolov8s.pt or yolov8m.pt for better accuracy

# # Start training
# model.train(
#     data='/kaggle/working/dataset.yaml',
#     epochs=30,
#     imgsz=416,
#     batch=16,
#     name='uec256_yolo_model',
#     project='food_nutrition_project'
# )

model = YOLO('yolov8n.pt')
model.train(
    data='/kaggle/working/dataset.yaml',
    epochs=20,
    imgsz=416,
    batch=16,
    name='uec256_kaggle',
    project='/kaggle/working'
)


Ultralytics 8.3.155 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=uec256_kaggle, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

train: Scanning /kaggle/working/dataset/labels/train... 31395 images, 0 backgrounds, 1 corrupt: 100%|██████████| 31395/31395 [00:21<00:00, 1466.45it/s]

train: /kaggle/working/dataset/images/train/150_37571.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.3262]
train: /kaggle/working/dataset/images/train/16_1581.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/16_1585.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/1_81.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/1_85.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/1_87.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/1_98.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/30_2902.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/35_3394.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/36_82.jpg: corrupt JPEG restored and saved
train: /kaggle/working/dataset/images/train/36_84.jpg: corrupt JPE

train: New cache created: /kaggle/working/dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 381.3±119.4 MB/s, size: 124.9 KB)


val: Scanning /kaggle/working/dataset/labels/val... 4737 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4737/4737 [00:03<00:00, 1410.05it/s]

...

      20/20      5.54G     0.7287      2.113      1.216          2        416: 100%|██████████| 1963/1963 [05:23<00:00,  6.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 149/149 [00:28<00:00,  5.30it/s]


                   all       4737       4737      0.484      0.473      0.481      0.372

20 epochs completed in 1.979 hours.
Optimizer stripped from /kaggle/working/uec256_kaggle/weights/last.pt, 16.7MB
Optimizer stripped from /kaggle/working/uec256_kaggle/weights/best.pt, 16.7MB

Validating /kaggle/working/uec256_kaggle/weights/best.pt...
Ultralytics 8.3.155 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 8,237,840 parameters, 0 gradients, 46.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 149/149 [00:27<00:00,  5.51it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       4737       4737      0.482      0.474       0.48      0.371
               class_0         92         92      0.303      0.663      0.392      0.294
               class_1         18         18      0.552      0.944      0.887      0.668
               class_2         17         17        0.4      0.412      0.468      0.274
               class_3         19         19      0.355      0.318      0.335      0.272
               class_4         18         18      0.436      0.722      0.566      0.416
               class_5         35         35      0.603      0.714      0.733      0.564
               class_6         30         30      0.661      0.433      0.602      0.445
               class_7         13         13      0.619      0.462      0.375      0.298
               class_8         29         29       0.54      0.759      0.698      0.466
               class_9         19         19      0.648      0.737       0.77       0.63
              class_1

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 1

In [ ]:
results_csv = '/kaggle/working/uec256_kaggle/results.csv'
df = pd.read_csv(results_csv)

plt.figure(figsize=(14, 5))

plt.subplot(1,2,1)
plt.plot(df['metrics/precision(B)'], label='Precision')
plt.plot(df['metrics/recall(B)'], label='Recall')
plt.plot(df['metrics/mAP50(B)'], label='mAP@0.5')
plt.plot(df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95')
plt.title("Validation Performance")
plt.xlabel("Epoch"); plt.ylabel("Score")
plt.legend()

plt.subplot(1,2,2)
plt.plot(df['train/box_loss'], label='Train Box Loss')
plt.plot(df['val/box_loss'], label='Val Box Loss')
plt.plot(df['train/cls_loss'], label='Train Cls Loss', linestyle='--')
plt.plot(df['val/cls_loss'], label='Val Cls Loss', linestyle='--')
plt.title("Loss Curves")
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
test_path = '/kaggle/working/dataset/images/test'
results = model.predict(
    source=test_path,
    save=True,
    save_txt=True,
    name='test_preds',
    project='/kaggle/working'
)

print("✅ Predictions saved in /kaggle/working/test_preds/")
